#Benchmark of Ministry of Information Technologies and Communications "100 Mil programadores" program

Data about the "100 mil programadores" program, each row is a students or an applicant.

Details of the data can be readed in [Misión TIC 100 mil programadores](https://www.datos.gov.co/Ciencia-Tecnolog-a-e-Innovaci-n/Misi-n-TIC-2020-100-mil-programadores/2emd-i46m)

- Import the libraries to make the benchmark

In [45]:
import requests
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import pyarrow.compute as pc
import json
import timeit
import os
import random

- Setup and download data

Set "download_data" to True if you want download the data from [Datos Gov](www.datos.gov.co) page directly.

The URL contain the query param to get all the data using the Socrata API (SODA API).
Details about the use of that API can be viewed in:

[SODA endpoints](https://dev.socrata.com/docs/endpoints.html)
[SODA Pagination](https://dev.socrata.com/docs/paging.html)

In [4]:
url_mintic2020_100mil = "https://www.datos.gov.co/resource/2emd-i46m.json?$limit=135000"

response = requests.get(url = url_mintic2020_100mil)

#TODO manage error

data = response.json()

## Save the data

Function declaration for each file

In [5]:
#Load df as "colums"
df = pd.DataFrame.from_records(data)
df.shape

(134563, 15)

In [7]:
def save_json(data):
    with open('./raw_data.json', 'w') as f:
        json.dump(data, f)
        
def save_df(df):
    df.to_pickle("./raw_data.pkl")
    
def save_parquet(df):
    table = pa.Table.from_pandas(df)
    pq.write_table(table, './data_mintic.parquet')

In [8]:
%timeit save_json(data)

2.07 s ± 25.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
%timeit save_df(df)

742 ms ± 8.13 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
%timeit save_parquet(df)

239 ms ± 6.76 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Read file

In [28]:
def read_json():
    with open('raw_data.json') as f:
        json_data = json.load(f)
    return json_data

In [66]:
%timeit read_json()

21.6 ns ± 0.595 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [19]:
%timeit pd.read_pickle("./raw_data.pkl")

237 ms ± 9.54 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
%timeit pq.read_table('./data_mintic.parquet')

17.3 ms ± 172 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Compare file sizes

In [12]:
size_json = os.path.getsize('./raw_data.json')
size_pkl = os.path.getsize('./raw_data.pkl') 
size_parquet = os.path.getsize('./data_mintic.parquet')
print(f'Size of the files are: {"{:.2f}".format(size_json/(1024*1024))} MB for json file, \
      {"{:.2f}".format(size_pkl/(1024*1024))} MB for pkl file and \
      {"{:.2f}".format(size_parquet/(1024*1024))} MB for parquet file.')

Size of the files are: 48.98 MB for json file,       18.57 MB for pkl file and       0.30 MB for parquet file.


# Number of rows

In order to perform operation over the data. The following aproach are taken:
- Operate over the python list for json data (Search over the .json is not a practical aproach and will take more time)
- Operate over the pandas dataframe for .pkl file.
- Operate ever a pyarrow table for .parquet file.

In [54]:
json_data = read_json()
df_data = pd.read_pickle("./raw_data.pkl")
parquet_data = pq.read_table('./data_mintic.parquet')

In [32]:
%timeit parquet_data.num_rows

49 ns ± 2.06 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [33]:
%timeit len(df_data)

455 ns ± 18.3 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [34]:
%timeit len(json_data)

67.6 ns ± 1.91 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


# Obtain a random element

In [46]:
random_index = random.randint(0,len(df_data))

In [56]:
%timeit parquet_data.take([random_index])

68.7 µs ± 1.22 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [57]:
%timeit df_data.iloc[random_index]

67.3 µs ± 2.98 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [58]:
%timeit json_data[random_index]

47.6 ns ± 1.95 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)
